In [1]:
import yfinance as yf
import pandas as pd
import mplfinance as mpf
import numpy as np

In [9]:
tickers = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN',
    'META', 'NVDA', 'SPY', 'QQQ'
]

periods = {
    'short': '1y',
    'mid': '3y',
    'long': '5y'
}

In [10]:
results = []

for ticker in tickers:
    for label, period in periods.items():
        data = yf.download(ticker, period=period, multi_level_index=False)

        a = compute_macd(data)
        b = generate_macd_events(a)
        c = events_to_position(b)

        bt, matrics = backtest_simple(c, cost_bps=5)
        bt, dd_rep, spell_d = drawdown_report(bt)
        perf = performance_report(bt, dd_report=dd_rep)

        results.append({
            'ticker': ticker,
            'horizon': label,
            'period': period,
            **perf
        })

res_df = pd.DataFrame(results)

C:\Users\HP\AppData\Local\Temp\ipykernel_13576\2625164394.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=period, multi_level_index=False)
[*********************100%***********************]  1 of 1 completed
C:\Users\HP\AppData\Local\Temp\ipykernel_13576\2625164394.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=period, multi_level_index=False)
[*********************100%***********************]  1 of 1 completed
C:\Users\HP\AppData\Local\Temp\ipykernel_13576\2625164394.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=period, multi_level_index=False)
[*********************100%***********************]  1 of 1 completed
C:\Users\HP\AppData\Local\Temp\ipykernel_13576\2625164394.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.downlo

In [13]:
res_df

,ticker,horizon,period,total_return,CAGR,ann_vol,Sharpe,max_drawdown,Calmar,n_periods
0,AAPL,short,1y,-0.157625,-0.158780,0.159391,-1.004021,-0.227809,-0.696990,250
1,AAPL,mid,3y,0.059606,0.019591,0.159889,0.201586,-0.260219,0.075287,752
2,AAPL,long,5y,0.495856,0.084220,0.174523,0.550934,-0.260218,0.323650,1255
3,MSFT,short,1y,-0.052507,-0.052916,0.160453,-0.261000,-0.140350,-0.377028,250
4,MSFT,mid,3y,0.062039,0.020375,0.160182,0.205426,-0.270952,0.075198,752
5,MSFT,long,5y,-0.079623,-0.016523,0.174809,-0.008023,-0.375438,-0.044009,1255
6,GOOGL,short,1y,0.335082,0.338172,0.249683,1.289946,-0.115770,2.921065,250
7,GOOGL,mid,3y,0.506903,0.147301,0.206394,0.768254,-0.153131,0.961927,752
8,GOOGL,long,5y,0.037594,0.007438,0.206863,0.138604,-0.452700,0.016430,1255
9,AMZN,short,1y,-0.142674,-0.143729,0.232653,-0.551885,-0.236232,-0.608422,250


In [12]:
order = ['short', 'mid', 'long']
res_df['horizon'] = pd.Categorical(
    res_df['horizon'],
    categories=order,
    ordered=True
)

In [26]:
res_df.groupby('horizon')[['Sharpe', 'Calmar']].describe()

C:\Users\HP\AppData\Local\Temp\ipykernel_13576\1118744542.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  res_df.groupby('horizon')[['Sharpe', 'Calmar']].describe()


Sharpe                                                              \
         count      mean       std       min       25%       50%       75%   
horizon                                                                      
short      8.0  0.018942  0.704599 -1.004021 -0.333721 -0.069965  0.303938   
mid        8.0  0.560900  0.406010  0.118876  0.204466  0.470874  0.845821   
long       8.0  0.363978  0.328057 -0.008023  0.172352  0.306228  0.416347   

                  Calmar                                                    \
              max  count      mean       std       min       25%       50%   
horizon                                                                      
short    1.289946    8.0  0.222150  1.167337 -0.696990 -0.434876 -0.096408   
mid      1.172782    8.0  0.520108  0.603273  0.012301  0.075265  0.235422   
long     1.062763    8.0  0.189408  0.263450 -0.044009  0.036089  0.101441   

                             
              75%       max  
horizon                      
short    0.245116  2.921065  
mid      0.879323  1.713518  
long     0.230210  0.774605

In [27]:
res_df.groupby('horizon')[['max_drawdown', 'CAGR']].describe()

C:\Users\HP\AppData\Local\Temp\ipykernel_13576\407137093.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  res_df.groupby('horizon')[['max_drawdown', 'CAGR']].describe()


max_drawdown                                                    \
               count      mean       std       min       25%       50%   
horizon                                                                  
short            8.0 -0.171289  0.050578 -0.236232 -0.225887 -0.153758   
mid              8.0 -0.239106  0.098527 -0.440994 -0.269574 -0.242474   
long             8.0 -0.378570  0.161877 -0.673681 -0.464622 -0.360304   

                             CAGR                                          \
              75%       max count      mean       std       min       25%   
horizon                                                                     
short   -0.134609 -0.115770   8.0  0.014092  0.160748 -0.158780 -0.075619   
mid     -0.156199 -0.136486   8.0  0.114703  0.138528  0.003310  0.020179   
long    -0.259465 -0.163762   8.0  0.073277  0.130319 -0.016523  0.012899   

                                       
              50%       75%       max  
horizon                                
short   -0.013141  0.048858  0.338172  
mid      0.041004  0.177827  0.375633  
long     0.034987  0.050141  0.387602

In [28]:
res_df.groupby('horizon')[['Sharpe','Calmar','max_drawdown','CAGR']].median()


C:\Users\HP\AppData\Local\Temp\ipykernel_13576\2502118272.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  res_df.groupby('horizon')[['Sharpe','Calmar','max_drawdown','CAGR']].median()


,Sharpe,Calmar,max_drawdown,CAGR
horizon,,,,
short,-0.069965,-0.096408,-0.153758,-0.013141
mid,0.470874,0.235422,-0.242474,0.041004
long,0.306228,0.101441,-0.360304,0.034987


In [29]:
res_df.sort_values(['horizon', 'Sharpe'])

,ticker,horizon,period,total_return,CAGR,ann_vol,Sharpe,max_drawdown,Calmar,n_periods
0,AAPL,short,1y,-0.157625,-0.158780,0.159391,-1.004021,-0.227809,-0.696990,250
9,AMZN,short,1y,-0.142674,-0.143729,0.232653,-0.551885,-0.236232,-0.608422,250
3,MSFT,short,1y,-0.052507,-0.052916,0.160453,-0.261000,-0.140350,-0.377028,250
21,QQQ,short,1y,-0.016530,-0.016661,0.125283,-0.071735,-0.150293,-0.110854,250
18,SPY,short,1y,-0.009545,-0.009621,0.086662,-0.068195,-0.117388,-0.081961,250
12,META,short,1y,0.019423,0.019580,0.218291,0.198663,-0.157222,0.124536,250
15,NVDA,short,1y,0.135537,0.136692,0.261334,0.619762,-0.225247,0.606856,250
6,GOOGL,short,1y,0.335082,0.338172,0.249683,1.289946,-0.115770,2.921065,250
10,AMZN,mid,3y,0.009911,0.003310,0.207800,0.118876,-0.269114,0.012301,752
1,AAPL,mid,3y,0.059606,0.019591,0.159889,0.201586,-0.260219,0.075287,752
